# SR Math Computations (forward only)
- Purpose: to connect their equations with their code and get overview
- loads sample data
- turns it into a list of session 'connection matrices'
- applies equations from the paper to forward propagate the sessions in the 'model'

In [1]:
import numpy as np
from helper_methods import *

import pickle
import torch
from torch import nn
from torch.nn import Module, Parameter
import torch.nn.functional as F

### Sample data from the paper's github
- in this folder also

In [2]:
train_data = pickle.load(open('sample' + '/train.txt', 'rb'))

In [3]:
valid_portion = .1
len(train_data[0])

1205

### Train data is list of sessions, which are list of clicked items in clicked order, and label data is the next clicked item held out from the train data

In [4]:
train_x, train_y = train_data
print(train_x[:5], '\n', train_y[:5])

[[1, 2], [1], [4], [6], [8, 9]] 
 [3, 2, 5, 7, 9]


In [5]:
n_samples = len(train_x)
sidx = np.arange(n_samples, dtype='int32')
np.random.shuffle(sidx)
n_train = int(np.round(n_samples * (1. - valid_portion)))

In [6]:
valid_set_x = [train_x[s] for s in sidx[n_train:]]
valid_set_y = [train_y[s] for s in sidx[n_train:]]
train_set_x = [train_x[s] for s in sidx[:n_train]]
train_set_y = [train_y[s] for s in sidx[:n_train]]

In [7]:
all_usr_pois = train_set_x
item_tail = [0] # for padding session with less < max clicks with 0's

In [8]:
us_lens = [len(upois) for upois in all_usr_pois] # length of all sessions

In [9]:
len_max = max(us_lens)
len_max, sum(us_lens) / len(all_usr_pois) # max session length, avg session length

(15, 2.809963099630996)

### Padding all sessions with 0, so all have len(session) == len_max

In [10]:
us_pois = [upois + item_tail * (len_max - le) for upois, le in zip(all_usr_pois, us_lens)]

In [11]:
# example of padding a sesssion
all_usr_pois[0] + [0] * (len_max - us_lens[0])

[49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [12]:
# mask with ones where session is non-zero
us_msks = [[1] * le + [0] * (len_max - le) for le in us_lens]

In [13]:
# example of mask
[1] * us_lens[0] + [0] * (len_max - us_lens[0])

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [14]:
# padded session, the corresponding mask
us_pois[0], us_msks[0]

([49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [15]:
# setting variables like in their code
inputs = np.asarray(us_pois)
mask = np.asarray(us_msks)
len_max = len_max
targets = np.asarray(train_set_y)
length = len(inputs)
shuffle = False
graph = None

In [16]:
n_node = 310 # is fixed from the sample data they proposed (number of items)
hidden_size = 100 # the magical hidden size, the d

In [17]:
# all sizes like from their default
input_size = hidden_size * 2
# they use 3 * hidden_size here (below), so they can split it up into tree parts when doing
# reset, update and newgate
gate_size = 3 * hidden_size 
w_ih = torch.Tensor(gate_size, input_size)
w_hh = torch.Tensor(gate_size, hidden_size)
b_ih = torch.Tensor(gate_size)
b_hh = torch.Tensor(gate_size)
b_iah = torch.Tensor(hidden_size)
b_oah = torch.Tensor(hidden_size)

In [18]:
# matrix multiplication with learnable weights: y = x*A^t + b
linear_edge_in = nn.Linear(hidden_size, hidden_size, bias=True)
linear_edge_out = nn.Linear(hidden_size, hidden_size, bias=True)
linear_edge_f = nn.Linear(hidden_size, hidden_size, bias=True)

In [19]:
batch_size = 100 # number of sessions in one training step
n_batch = int(length / batch_size)
 
if length % batch_size != 0: #edge case for last batch
    n_batch += 1

In [20]:
slices = np.split(np.arange(n_batch * batch_size), n_batch) # indices for each batch

In [21]:
slices[-1] = slices[-1][:(length - batch_size * (n_batch - 1))] # edge case for last batch

## Batch 1

In [22]:
b1_slice = slices[0] # we only do forward step for first batch

In [23]:
inputs_b1, mask_b1, targets_b1 = inputs[b1_slice], mask[b1_slice], targets[b1_slice]

In [24]:
# items: will be sessions padded with 0's until len(session) = max_len of the sessions in the batch 
# n_node_b1: list with len of each session in batch
# A: will be list of all the connection matrices
# alias_inputs: will be list for each session to map back to the original click order
items, n_node_b1, A, alias_inputs = [],[],[],[] 

In [25]:
for u_input in inputs_b1:
            n_node_b1.append(len(np.unique(u_input)))

In [26]:
max_n_node = np.max(n_node_b1)

### Creation of all the connection matrices for batch 1

In [27]:
# for each session in batch
for u_input in inputs_b1:
    node = np.unique(u_input) # get unique items in session (items can be clicked mutiple times)
    items.append(node.tolist() + ((max_n_node - len(node)) * [0])) # append with 0's until max len in batch

    u_A = np.zeros((max_n_node, max_n_node)) # initialize the matrix with 0s (out degree only at first)

    for i in np.arange(len(u_input) - 1): # fill out the matrix
        if u_input[i + 1] == 0: # if next item is 0, we know there are not anymore clicks, so break
            break
        u = np.where(node == u_input[i])[0][0] # get row entry for current item i
        v = np.where(node == u_input[i + 1])[0][0] # get col entry for next item i + 1
        u_A[u][v] = 1 # mark outgoing edge from u -> v 

    # sum over ingoing edges to normalize as described in paper, 1 / sum(outgoing edges)
    u_sum_in = np.sum(u_A, 0) 
    u_sum_in[np.where(u_sum_in == 0)] = 1 # divide 0 edge case
    u_A_in = np.divide(u_A, u_sum_in)

    # As above: for outgoing, hence across rows
    u_sum_out = np.sum(u_A, 1)
    u_sum_out[np.where(u_sum_out == 0)] = 1
    u_A_out = np.divide(u_A.transpose(), u_sum_out)
    
    # concatenate in_degree and out_degree and then tramspose, because concat defaults to rows
    # so get: max_len X 2*max_len Matrix as from paper
    u_A = np.concatenate([u_A_in, u_A_out]).transpose()

    A.append(u_A)
    alias_inputs.append([np.where(node == i)[0][0] for i in u_input]) 

### Example of a connection matrix

In [28]:
# first session padded with 0's
print('Original session:', u_input,'\n'
      'Adjusted so max len is max of batch:', items[-1], '\n',
      'The mapping from the adjusted into original click order:', alias_inputs[-1],'\n')

print('Ingoing connection matrix (click order is from u_input, \n while matrix is asc order):')
print(A[-1][:,:max_n_node])
print('Outgoing matrix part:')
print(A[-1][:,max_n_node:])
print('Full conncection matrix:',A[-1].shape)

Original session: [45  0  0  0  0  0  0  0  0  0  0  0  0  0  0] 
Adjusted so max len is max of batch: [0, 45, 0, 0, 0, 0] 
 The mapping from the adjusted into original click order: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 

Ingoing connection matrix (click order is from u_input, 
 while matrix is asc order):
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Outgoing matrix part:
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Full conncection matrix: (6, 12)


## Model forwarding computations

In [29]:
alias_inputs = torch.Tensor(alias_inputs).long()
items = torch.Tensor(items).long()
A = torch.Tensor(A).float()
mask = torch.Tensor(mask).long()

### Hidden Embedding
- each item (session vector) which is vector with dim of max_len of clicks in batch will be converted to a max_len X hidden_size Matrix representation

In [30]:
embedding = nn.Embedding(n_node, hidden_size) 
hidden = embedding(items)
items.shape, hidden.shape

(torch.Size([100, 6]), torch.Size([100, 6, 100]))

### GNN Hidden - Equation 1 from paper

In [31]:
input_in = torch.matmul(A[:, :, :A.shape[1]], linear_edge_in(hidden)) + b_iah
input_out = torch.matmul(A[:, :, A.shape[1]:2*A.shape[1]], linear_edge_in(hidden)) + b_iah

In [32]:
inputs_gnn = torch.cat([input_in, input_out], 2)

In [33]:
inputs_gnn.shape

torch.Size([100, 6, 200])

### GNN Gate Computations

In [34]:
gate_in =  F.linear(inputs_gnn, w_ih, b_ih)
gate_hidden = F.linear(hidden, w_hh, b_hh) 
gate_in.shape, gate_hidden.shape

(torch.Size([100, 6, 300]), torch.Size([100, 6, 300]))

In [35]:
# they split the 3*hidden_size into three chunks
# i_r = input | reset gate weights
# i_i = input | input gate weights
# i_n = input | newgate weights 
# h_x = hidden | x...
i_r, i_i, i_n = gate_in.chunk(3, 2)
h_r, h_i, h_n = gate_hidden.chunk(3, 2)

In [36]:
resetgate = torch.sigmoid(i_r + h_r) # Equation 3 from paper
inputgate = torch.sigmoid(i_i + h_i) # Equation 2 from paper
newgate = torch.tanh(i_n + resetgate * h_n) # Equation 4 from paper
hy = newgate + inputgate * (hidden - newgate) # ask Peter, how does this match with Equation 5 from paper

### Attention, Global session, Hybrid Session computations

In [37]:
get = lambda i: hidden[i][alias_inputs[i]] # function to get orginal click order (sequential) in hidden repr.

In [38]:
seq_hidden = torch.stack([get(i) for i in torch.arange(len(alias_inputs)).long()])

### Score computations